In [2]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from functools import reduce

In [111]:
def get_labels(house_num, PATH='data/low_freq/'):
    return pd.read_table("{0}house_{1}/labels.dat".format(PATH, house_num), header=None, delim_whitespace=True, 
                               names=['channel', 'appliance'], dtype={'channel': np.int16, 'appliance': np.str})
    

In [115]:
labels_house_1 = get_labels(house_num=1)

In [116]:
labels_house_1.keys()

Index(['channel', 'appliance'], dtype='object')

In [117]:
labels_house_1

,channel,appliance
0,1,mains
1,2,mains
2,3,oven
3,4,oven
4,5,refrigerator
5,6,dishwaser
6,7,kitchen_outlets
7,8,kitchen_outlets
8,9,lighting
9,10,washer_dryer


<p>Channels Appliance </p>
<p>5 Refrigerator</p>
<p>6 Dishwasher</p>
<p>11 Microwave</p>
<p>13 Electric Heater</p>

In [30]:
#channel_num = 5
#PATH = "data/low_freq/house_1/channel_"+str(channel_num)+'.dat'
#refrigerator_data = pd.read_table(PATH, header=None, delim_whitespace=True, 
#             names=['time_stamp', 'power'], dtype={'time_stamp': np.string_, 'power': np.float128})

In [130]:
def get_channel_data(channel_num, as_csv=False):
    '''
    input: channel_num (int)
            as_csv (bool)
    return: 
        True if as_csv=True
        pandas.core.frame.DataFrame if as_csv=False
    '''
    PATH = "data/low_freq/house_1/channel_"+str(channel_num)+'.dat'
    if as_csv:
        ret_file = labels_house_1.loc[labels_house_1['channel'] == channel_num].iloc[0][1]
        pd.read_table(PATH, header=None, delim_whitespace=True, 
                        names=['time_stamp', 'power'], 
                             dtype={'time_stamp': np.string_, 'power': np.float128}).to_csv(ret_file+'_data.csv')
        return True 
    else:
        return pd.read_table(PATH, header=None, delim_whitespace=True, 
                        names=['time_stamp', 'power_{0}'.format(labels_house_1.loc[labels_house_1['channel'] == channel_num].iloc[0][1])], 
                             dtype={'time_stamp': np.string_, 'power': np.float128})        

In [4]:
ch_nums = [5, 6, 11, 13, 14]

In [48]:
for ch in ch_nums:
    get_channel_data(ch, as_csv=True)

In [75]:
def merge_all(ch_nums):
    full_data = get_channel_data(channel_num=ch_nums[0])
    for ch in ch_nums[1:]:
        full_data = full_data.merge(get_channel_data(ch), how='inner', on=['time_stamp'])

In [133]:
full_data = merge_all(ch_nums)

In [135]:
full_data.keys()

Index(['time_stamp', 'power_refrigerator', 'power_dishwaser',
       'power_microwave', 'power_electric_heat', 'power_stove'],
      dtype='object')

<p>Need 2 csvs
<p>1: Need day by day data for each appliance - num_days x num_devices
<p>2: Need hour by hour data for each day for each appliance - 

In [137]:
full_data['time_stamp'] = pd.to_datetime(full_data['time_stamp'], unit='s')

In [139]:
full_data.head()

,time_stamp,power_refrigerator,power_dishwaser,power_microwave,power_electric_heat,power_stove
0,2011-04-18 13:22:13,6.0,0.0,5.0,0.0,0.0
1,2011-04-18 13:22:16,6.0,0.0,5.0,0.0,0.0
2,2011-04-18 13:22:20,6.0,0.0,5.0,0.0,0.0
3,2011-04-18 13:22:23,6.0,1.0,5.0,0.0,0.0
4,2011-04-18 13:22:26,6.0,0.0,5.0,0.0,0.0


In [125]:
fd.resample(rule='H', on='time_stamp').sum()

,power_refrigerator,power_dishwaser,power_microwave,power_electric_heat,power_stove
time_stamp,,,,,
2011-04-18 13:00:00,3563.0,113.0,2662.0,522.0,436.0
2011-04-18 14:00:00,134014.0,425.0,27448.0,105.0,255.0
2011-04-18 15:00:00,116762.0,8.0,75297.0,318.0,265.0
2011-04-18 16:00:00,54328.0,10.0,3732.0,0.0,0.0
2011-04-18 17:00:00,46438.0,8.0,3620.0,0.0,0.0
2011-04-18 18:00:00,46377.0,5.0,25930.0,168.0,115.0
2011-04-18 19:00:00,60233.0,24.0,3935.0,52.0,3.0
2011-04-18 20:00:00,55216.0,22.0,75016.0,322.0,276.0
2011-04-18 21:00:00,64486.0,16.0,3976.0,0.0,1.0


In [140]:
day_by_day = full_data.resample(rule='D', on='time_stamp').sum()

In [142]:
day_by_day.head()

,power_refrigerator,power_dishwaser,power_microwave,power_electric_heat,power_stove
time_stamp,,,,,
2011-04-18,643334.0,1117.0,496669.0,2872.0,2376.0
2011-04-19,1087604.0,1107430.0,339651.0,1991.0,1824.0
2011-04-20,1176889.0,1237.0,623093.0,3426.0,3275.0
2011-04-21,951020.0,958.0,309471.0,1653.0,1527.0
2011-04-22,1156480.0,1331.0,205417.0,1655.0,1639.0


In [150]:
hour_by_hour = full_data.resample(rule='H', on='time_stamp').sum()

In [152]:
hour_by_hour.head()

,power_refrigerator,power_dishwaser,power_microwave,power_electric_heat,power_stove
time_stamp,,,,,
2011-04-18 13:00:00,3563.0,113.0,2662.0,522.0,436.0
2011-04-18 14:00:00,134014.0,425.0,27448.0,105.0,255.0
2011-04-18 15:00:00,116762.0,8.0,75297.0,318.0,265.0
2011-04-18 16:00:00,54328.0,10.0,3732.0,0.0,0.0
2011-04-18 17:00:00,46438.0,8.0,3620.0,0.0,0.0


In [157]:
refrigerator_hh = hour_by_hour[['power_refrigerator']]
dishwasher_hh = hour_by_hour[['power_dishwaser']]
microwave_hh = hour_by_hour[['power_microwave']]
electric_heat_hh = hour_by_hour[['power_electric_heat']]
stove_hh = hour_by_hour[['power_stove']]

In [166]:
#to_csv function
def to_csv(df_list):
    for df in df_list:
        df.to_csv(df.name + '.csv')

In [169]:
day_by_day.name = 'day_by_day'
hour_by_hour.name = 'hour_by_hour'
refrigerator_hh.name = 'refrigerator_hh'
dishwasher_hh.name = 'dishwasher_hh'
microwave_hh.name = 'microwave_hh'
electric_heat_hh.name = 'electric_heat_hh'
stove_hh.name = 'stove_hh'

In [171]:
df_list = [day_by_day, hour_by_hour, refrigerator_hh, dishwasher_hh, microwave_hh, electric_heat_hh, stove_hh]

In [172]:
to_csv(df_list)